this code for compare diffrent types of feature selection techniques.
experiment 2: compare feature selection methods following the initial use of the univariate ROC-AUC method, including
two embedded techniques: LASSO and Random forest variable importance. Moreover, mutual information and ReliefF methods. 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV, cross_val_score, train_test_split, LeaveOneOut, KFold, StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, classification_report
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression, Lasso
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier, GradientBoostingClassifier, ExtraTreesClassifier
import xgboost as xgb
from feature_engine.selection import DropDuplicateFeatures, DropConstantFeatures
from sklearn.feature_selection import SelectFromModel, VarianceThreshold
from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN, BorderlineSMOTE, SVMSMOTE
from sklearn.ensemble import VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from mlxtend.classifier import EnsembleVoteClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score, plot_roc_curve
from yellowbrick.classifier import ROCAUC


# input the data (gene expression array) and labels

In [2]:
filenames = ['riaz', 'gide', 'van allen', 'hugo', 'lee']
name= filenames[1]
name

'gide'

In [3]:
filenames = ['riaz', 'gide', 'van allen', 'hugo', 'lee']
name= filenames[1]

# Read logged data
logged_data = pd.read_csv('data/' + name + '_logged_data.csv', index_col='Unnamed: 0').T
print(logged_data.shape)
print(logged_data.head())

# Read labels
label = pd.read_csv('data/' + name + '.Pre.Samples.corr.csv', index_col='Unnamed: 0')
print(label.shape)
print(label.head())

# Count of labels
print(label['Resp_NoResp'].value_counts())

# Normalized count of labels
print(label['Resp_NoResp'].value_counts(normalize=True))

# Label encoding
y = label.replace({'No_Response': 0, 'Response': 1})
labels = ['No_Response', 'Response']  # for plotting convenience later on

print(y.head())
print(y['Resp_NoResp'].head())
print(y.iloc[:, 0].head())
print(y.iloc[:, 0].shape)


(72, 22098)
            FAM138A  OR4F5    WASH9P  LINC01409    FAM87B  LINC01128  \
PD1_2_PRE       0.0    0.0  5.076753   3.201590  0.000000   1.952068   
PD1_4_PRE       0.0    0.0  4.458884   4.227967  2.116873   3.194789   
PD1_6_PRE       0.0    0.0  4.077774   3.503593  0.000000   2.612036   
PD1_15_PRE      0.0    0.0  5.344685   3.615239  0.000000   3.172880   
PD1_16_PRE      0.0    0.0  4.655988   3.862449  0.000000   1.971780   

            LINC00115    FAM41C  LINC02593    SAMD11  ...    SERF1A     SFTA3  \
PD1_2_PRE    6.669880  0.000000   2.033235  1.963940  ...  0.000000  0.000000   
PD1_4_PRE    7.434516  0.551825   0.000000  1.995952  ...  0.299067  0.000000   
PD1_6_PRE    8.221054  0.000000   0.000000  0.725464  ...  0.000000  0.000000   
PD1_15_PRE   8.755374  0.000000   0.000000  1.968548  ...  0.000000  0.000000   
PD1_16_PRE   7.602921  0.000000   0.000000  2.722831  ...  0.000000  0.414746   

            SLCO1B7      SMN1   SPATA13  ST6GALNAC6      TAB2    TBC

In [4]:
#read the immune related genes from immport
IRG= pd.read_csv('IRG.csv')
print(IRG.shape)
IRG.head()

(1811, 1)


,Symbol
0,AZGP1
1,B2M
2,CALR
3,CANX
4,CD1A


# data oversampling

In [5]:
# Define features (X) and target (Y)
X = logged_data
Y = y.iloc[:, 0]

# Oversampling using BorderlineSMOTE
ada = BorderlineSMOTE(
    sampling_strategy='auto',  # samples only the minority class
    random_state=0,  # for reproducibility
    k_neighbors=5,
    m_neighbors=10,
    kind='borderline-1',
    n_jobs=4
)

# Resample the data
X_res, y_res = ada.fit_resample(X, Y)

# Print the shapes of resampled data
print(X_res.shape, y_res.shape)

# Print the counts of the original and resampled target variable
print(Y.value_counts(), y_res.value_counts())


C:\Users\LENOVO\anaconda3\lib\site-packages\imblearn\over_sampling\_smote\filter.py:192: FutureWarning: The parameter `n_jobs` has been deprecated in 0.10 and will be removed in 0.12. You can pass an nearest neighbors estimator where `n_jobs` is already set instead.
  warnings.warn(


(90, 22098) (90,)
1    45
0    27
Name: Resp_NoResp, dtype: int64 1    45
0    45
Name: Resp_NoResp, dtype: int64


# separate dataset into train and test


In [6]:
# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.3, random_state=0)

# Display shapes of train and test sets
print(X_train.shape, X_test.shape)

# Display first few rows of test set
print(X_test.head())

# Display normalized value counts of target variable in test set
print(y_test.value_counts(normalize=True))

# Keep a copy of the original datasets
X_train_original = X_train.copy()
X_test_original = X_test.copy()

# Check shapes of train and test sets after copying
print(X_train.shape, X_test.shape)


(63, 22098) (27, 22098)
    FAM138A  OR4F5    WASH9P  LINC01409  FAM87B  LINC01128  LINC00115  \
2       0.0    0.0  4.077774   3.503593     0.0   2.612036   8.221054   
13      0.0    0.0  5.103238   3.712569     0.0   2.741953   7.340254   
53      0.0    0.0  5.786494   3.364437     0.0   1.642540   6.363859   
41      0.0    0.0  5.149037   1.804976     0.0   1.534749   6.492433   
66      0.0    0.0  5.662066   3.522241     0.0   1.989737   5.259296   

      FAM41C  LINC02593    SAMD11  ...    SERF1A  SFTA3  SLCO1B7      SMN1  \
2   0.000000        0.0  0.725464  ...  0.000000    0.0      0.0  5.472809   
13  0.000000        0.0  2.411625  ...  0.000000    0.0      0.0  4.958523   
53  0.000000        0.0  1.166203  ...  0.781725    0.0      0.0  4.865167   
41  0.000000        0.0  2.738410  ...  0.000000    0.0      0.0  4.066993   
66  0.513524        0.0  0.936701  ...  1.733343    0.0      0.0  5.379111   

     SPATA13  ST6GALNAC6      TAB2    TBC1D3      TUT1   USP17L8  
2

# use of the data selected by Univariate ROC-AUC Performance to begin with experiment 2

In [ ]:
X_train = pd.read_csv('gide_X_train_uni_roc-auc.csv',index_col = 'Unnamed: 0')
print(X_train_uni.shape)
X_train_uni.head()


In [ ]:
X_test = pd.read_csv('gide_X_test_uni_roc-auc.csv',index_col = 'Unnamed: 0')
print(X_test_uni.shape)
X_test_uni.head()

In [37]:
# keep acopy of the data selected by Univariate ROC-AUC Performance
X_train_uni = X_train
X_test_uni = X_test

In [33]:
X_train_uni= pd.read_csv('gide_X_train_SOMTE bord_uni XGB_corr.csv',index_col = 'Unnamed: 0')
print(X_train_uni.shape)
X_train_uni.head()

(63, 1141)


,MRPL20,PLCH2,CEP104,VAMP3,PIK3CD,PIK3CD-AS1,CTNNBIP1,DFFA,TMEM51-AS1,PADI3,...,PAK3,MCTS1,ZNF449,ADGRG4,FGF13,SPANXB1,GPR50,DDX3Y,LINC01238,PLA2G2C
79,7.570243,1.821964,5.856766,5.932988,5.742181,5.034410,1.973813,5.702246,0.973665,0.042752,...,0.151941,5.211416,5.415581,0.082789,0.137672,4.049688,0.0,8.110795,0.253986,0.274204
82,7.093483,1.173003,5.383863,5.930621,5.519404,5.176408,1.545476,5.867246,1.028031,1.085278,...,0.593874,4.935200,5.107300,0.152173,0.153646,2.556665,0.0,8.199987,0.171713,0.000000
3,7.192808,5.522151,5.474071,5.522044,4.459219,6.050044,2.908931,5.054147,1.955722,0.000000,...,0.979523,4.647634,5.297447,0.289800,0.390714,0.000000,0.0,1.632556,1.851140,0.000000
62,7.382451,2.226904,5.904765,5.805884,5.167068,7.019796,1.433969,5.277627,0.654643,0.000000,...,0.577084,4.519005,5.120259,0.000000,0.387281,2.410704,0.0,0.000000,0.000000,0.000000
75,7.679880,1.795303,5.313456,5.910707,5.403834,4.078740,2.384913,5.637014,0.933163,0.000000,...,1.028568,4.929844,5.521676,0.275970,0.138589,0.000000,0.0,4.296515,0.794905,0.000000


In [34]:
X_train =X_train_uni

In [35]:
X_test = pd.DataFrame(X_test_original,index= X_test_original.index, columns=X_train_uni.columns)
X_test.head()

,MRPL20,PLCH2,CEP104,VAMP3,PIK3CD,PIK3CD-AS1,CTNNBIP1,DFFA,TMEM51-AS1,PADI3,...,PAK3,MCTS1,ZNF449,ADGRG4,FGF13,SPANXB1,GPR50,DDX3Y,LINC01238,PLA2G2C
2,7.218077,2.293300,4.915940,6.239549,5.017539,6.953420,0.724536,5.186254,0.977094,0.000000,...,0.067190,4.418221,4.760017,0.000000,0.208344,0.000000,0.0,8.503465,0.000000,0.000000
13,7.701680,1.346181,6.306751,6.384332,5.106100,5.979453,2.051514,5.993600,1.451240,0.000000,...,0.705292,5.000908,5.341430,0.253917,0.102159,0.000000,0.0,8.616417,0.000000,1.024800
53,7.399529,2.436839,5.906438,5.848469,5.699680,6.801373,3.114382,5.954255,2.461485,0.000000,...,0.462820,4.764817,5.292321,0.198839,0.098196,0.000000,0.0,8.314458,0.000000,2.062846
41,7.166596,1.919200,4.380314,5.481264,4.155765,0.000000,2.317842,4.958538,0.951823,0.000000,...,0.833379,3.943569,3.836622,0.000000,0.401814,0.000000,0.0,5.582638,1.572735,0.000000
66,7.523935,2.680563,5.882695,6.348137,6.085321,7.512046,1.727732,5.812483,2.205740,0.515895,...,0.565798,4.631293,5.543352,0.000000,0.718821,2.961867,0.0,0.000000,1.101279,0.552874


In [36]:
X_train.columns

Index(['MRPL20', 'PLCH2', 'CEP104', 'VAMP3', 'PIK3CD', 'PIK3CD-AS1',
       'CTNNBIP1', 'DFFA', 'TMEM51-AS1', 'PADI3',
       ...
       'PAK3', 'MCTS1', 'ZNF449', 'ADGRG4', 'FGF13', 'SPANXB1', 'GPR50',
       'DDX3Y', 'LINC01238', 'PLA2G2C'],
      dtype='object', length=1141)

In [24]:
IRG_feat=X_train.columns.intersection(IRG['Symbol'])
IRG_feat

Index(['PIK3CD', 'CSF3R', 'IL12RB2', 'RFX5', 'ADAR', 'SH2D1B', 'XCL2', 'XCL1',
       'ADIPOR1', 'CTSE',
       ...
       'PROCR', 'PLTP', 'MX1', 'TMPRSS6', 'APOBEC3H', 'ADM2', 'IL3RA', 'CXCR3',
       'PAK3', 'FGF13'],
      dtype='object', length=167)

# relieff

In [12]:
X_train.shape, X_test.shape

((63, 1141), (27, 1141))

In [13]:
from skrebate import ReliefF
fs = ReliefF()
fs.fit(X_train.values, y_train.values)


ReliefF()

In [ ]:
from sklearn.ensemble import VotingClassifier
accuracy_rate = []
clf1 = ExtraTreesClassifier()
clf2 = LogisticRegression(penalty ='l2', C=0.1, solver='liblinear', random_state=4,max_iter = 1000)
clf3 = SVC(gamma =1e-05, probability = True, decision_function_shape = 'ovo', kernel = 'linear')
clf4 = AdaBoostClassifier()
clf5= XGBClassifier()


# Will take some time
for i in range(5,300,5):
    
    indx_sort=fs.feature_importances_.argsort()[-i:][::-1]
    X_train_rf= X_train.iloc[:,indx_sort]
    X_test_rf= X_test.iloc[:,indx_sort]
    
    eclf1 = VotingClassifier(estimators=[ ('logistic', clf2),('SVM', clf3),
                                           ('XGB', clf5), 
                                          ('extratree',clf1), ('AdaBoost', clf4)
                                        ], voting='hard') 
    
   
    print(i)
    
    
    for clf in (clf1,clf2, clf3, clf4, clf5, eclf1):
        clf.fit(X_train_rf, y_train)
        y_pred = clf.predict(X_test_rf)
        print(clf.__class__.__name__, round(accuracy_score(y_test, y_pred),3))
        
  
    # Number of iterations
    num_iterations = 3

    # Initialize a list to store accuracy scores
    ensemble_scores = []

    # Perform multiple iterations of the VotingClassifier
    for _ in range(num_iterations):
        eclf1.fit(X_train_rf, y_train)
        predictions = eclf1.predict(X_test_rf)
        ensemble_acc = accuracy_score(y_test, predictions)
        ensemble_scores.append(ensemble_acc)

    print("Ensemble Voting Accuracy:", ensemble_scores)
    
    # Calculate the mean accuracy of the ensemble
    mean_ensemble_accuracy = np.mean(ensemble_scores)
    print("Mean Ensemble Voting Accuracy:", mean_ensemble_accuracy)

    print(X_train_rf.columns)
    accuracy_rate.append(mean_ensemble_accuracy)
    

In [ ]:
accuracy_rate

In [ ]:
accuracy_plot = pd.DataFrame()
accuracy_plot['ReliefF_accuracy'] = accuracy_rate

In [ ]:
plt.figure(figsize=(20,6))
plt.plot(range(5,300,5),accuracy_rate,color='blue', linestyle='solid', marker='o',
         markerfacecolor='blue', markersize=10)

title =plt.title('ReliefF Accuracy vs. No. of Genes', fontsize=20)
plt.xlabel('No. of Genes', fontsize=16)
plt.ylabel('ReliefF Accuracy', fontsize=16)


In [14]:
indx_sort=fs.feature_importances_.argsort()[-95:][::-1]
indx_sort

array([ 435,  741,  744,  729,  761,  735,  725,  670,  437,  754, 1126,
        792,  730,  723,  443,  756, 1073,  340,  727,  721,  442,  726,
        434,  426,  436,  926,  438,  742,  753,  431,  732,  733,  719,
        728,  253,  743,  441,  949,  610,  734,  736,  388,  344,  439,
        715,  389,  158,  354, 1031,   81,  429,  748,  738,  750,  430,
        717,  522, 1021,  561,  206,  774,  751,  502,  714,  638,  680,
        440, 1060,  602,  593,   79,  390,  647,  325,  433,  830,  634,
        604,   80,  787, 1022,  755,  606,  428,  927,  762,  838,  901,
        222,  368,  162, 1042,  533,  791,  731], dtype=int64)

In [15]:
X_train_rf= X_train.iloc[:,indx_sort]
X_train_rf.shape

(63, 95)

In [16]:
X_test_rf= X_test.iloc[:,indx_sort]
X_test_rf.shape

(27, 95)

In [17]:
X_train_rf.columns

Index(['TRBV7-6', 'TRAJ52', 'TRAJ47', 'TRAV24', 'TRAJ7', 'TRAV38-1', 'TRAV20',
       'IFNG', 'TRBV6-8', 'TRAJ28', 'CXCR3', 'LINC02312', 'TRAV25', 'TRAV16',
       'TRBV27', 'TRAJ24', 'TMPRSS3', 'UBD', 'TRAV22', 'TRAV13-2', 'TRBV24-1',
       'TRAV21', 'TRBV5-5', 'TRBV3-1', 'TRBV5-6', 'TBX21', 'TRBV5-7', 'TRAJ50',
       'TRAJ29', 'TRBV6-5', 'TRAV29DV5', 'TRAV30', 'TRAV8-2', 'TRAV23DV6',
       'CXCL11', 'TRAJ49', 'TRBV23-1', 'CD7', 'P2RY6', 'TRAV26-2',
       'TRAV38-2DV8', 'AOAH-IT1', 'HLA-A', 'TRBV7-9', 'TRAV5', 'TRGJ2', 'ICOS',
       'PTCRA', 'NCR1', 'SLAMF7', 'TRBV9', 'TRAJ35', 'TRAV41', 'TRAJ32',
       'TRBV11-2', 'TRAV10', 'DBH-AS1', 'NKG7', 'IFITM1', 'DTX3L', 'GCH1',
       'TRAJ31', 'SHC3', 'TRAV3', 'KLRC4', 'OAS2', 'TRBV10-3', 'ZNF831',
       'BATF2', 'SMTNL1', 'SLAMF6', 'TRGJ1', 'CNTN1', 'ZNF354C', 'TRBV6-6',
       'HAPLN3', 'CD27', 'CTSW', 'SLAMF1', 'BATF', 'SIGLEC8', 'TRAJ27',
       'GPR152', 'TRBV7-3', 'SKAP1', 'TRAJ1', 'SYNGR3', 'TBC1D28', 'LAMP3',
       'TNFAIP3',

In [ ]:
IRG_feat=X_train_rf.columns.intersection(IRG['Symbol'])
IRG_feat

In [18]:
X_train = X_train_rf
X_test= X_test_rf


In [19]:
#save a copy of reliefF result for the following experiment 
X_train.to_csv('gide_X_train_reliefF.csv')
X_test.to_csv('gide_X_test_reliefF.csv')

# Mutual information

In [ ]:
# use of the data selected by Univariate ROC-AUC Performance
X_train = X_train_uni
X_test = X_test_uni

In [ ]:
# ensure the sizes of X_train and X_test.
X_train.shape, X_test.shape

In [ ]:
# to obtain the mutual information values
from sklearn.feature_selection import mutual_info_classif, mutual_info_regression

# to select the features
from sklearn.feature_selection import SelectKBest, SelectPercentile

In [ ]:
from sklearn.ensemble import VotingClassifier
accuracy_rate = []
clf1 = ExtraTreesClassifier()
clf2 = LogisticRegression(penalty ='l2', C=0.1, solver='liblinear', random_state=4,max_iter = 1000)
clf3 = SVC(gamma =1e-05, probability = True, decision_function_shape = 'ovo', kernel = 'linear')
clf4 = AdaBoostClassifier()
clf5= XGBClassifier()

# Will take some time
for i in range(5,300,5):
    
    sel_ = SelectKBest(mutual_info_classif, k=i).fit(X_train, y_train)
    # display features
    best_feature=X_train.columns[sel_.get_support()]
    X_train_m = sel_.transform(X_train)
    X_test_m = sel_.transform(X_test)
    X_train_m = pd.DataFrame(X_train_m,index= X_train_original.index, columns=best_feature)
    X_test_m = pd.DataFrame(X_test_m,index= X_test_original.index, columns=best_feature)
   
    
    eclf1 = VotingClassifier(estimators=[ ('logistic', clf2),('SVM', clf3),
                                           ('XGB', clf5), 
                                          ('extratree',clf1), ('AdaBoost', clf4)
                                        ], voting='hard')    
    print(i)
    
    for clf in (clf1,clf2, clf3, clf4, clf5, eclf1):
        clf.fit(X_train_m, y_train)
        y_pred = clf.predict(X_test_m)
        #df_test[clf.__class__.__name__] = y_pred
        print(clf.__class__.__name__, round(accuracy_score(y_test, y_pred),3))
    
    # Number of iterations
    num_iterations = 3

    # Initialize a list to store accuracy scores
    ensemble_scores = []

    # Perform multiple iterations of the VotingClassifier
    for _ in range(num_iterations):
        eclf1.fit(X_train_m, y_train)
        predictions = eclf1.predict(X_test_m)
        ensemble_acc = accuracy_score(y_test, predictions)
        ensemble_scores.append(ensemble_acc)

    print("Ensemble Voting Accuracy:", ensemble_scores)
    
    # Calculate the mean accuracy of the ensemble
    mean_ensemble_accuracy = np.mean(ensemble_scores)
    print("Mean Ensemble Voting Accuracy:", mean_ensemble_accuracy)

    print(X_train_m.columns)
    accuracy_rate.append(mean_ensemble_accuracy)

In [ ]:
accuracy_rate

In [ ]:
plt.figure(figsize=(20,6))
plt.plot(range(5,300,5),accuracy_rate,color='blue', linestyle='dashed', marker='o',
         markerfacecolor='red', markersize=10)
plt.title('Mutual Info Accuracy vs. No. of Genes')
plt.xlabel('No. of Genes')
plt.ylabel('Mutual Info')

In [ ]:
ccuracy_plot = pd.DataFrame()
accuracy_plot['Mutual info'] = accuracy_rate

# lasso

In [ ]:
# use of the data selected by Univariate ROC-AUC Performance
X_train = X_train_uni
X_test = X_test_uni

In [38]:
# ensure the sizes of X_train and X_test.
X_train.shape, X_test.shape

((63, 1141), (27, 1141))

In [39]:
X_train_uni.head()

,MRPL20,PLCH2,CEP104,VAMP3,PIK3CD,PIK3CD-AS1,CTNNBIP1,DFFA,TMEM51-AS1,PADI3,...,PAK3,MCTS1,ZNF449,ADGRG4,FGF13,SPANXB1,GPR50,DDX3Y,LINC01238,PLA2G2C
79,7.570243,1.821964,5.856766,5.932988,5.742181,5.034410,1.973813,5.702246,0.973665,0.042752,...,0.151941,5.211416,5.415581,0.082789,0.137672,4.049688,0.0,8.110795,0.253986,0.274204
82,7.093483,1.173003,5.383863,5.930621,5.519404,5.176408,1.545476,5.867246,1.028031,1.085278,...,0.593874,4.935200,5.107300,0.152173,0.153646,2.556665,0.0,8.199987,0.171713,0.000000
3,7.192808,5.522151,5.474071,5.522044,4.459219,6.050044,2.908931,5.054147,1.955722,0.000000,...,0.979523,4.647634,5.297447,0.289800,0.390714,0.000000,0.0,1.632556,1.851140,0.000000
62,7.382451,2.226904,5.904765,5.805884,5.167068,7.019796,1.433969,5.277627,0.654643,0.000000,...,0.577084,4.519005,5.120259,0.000000,0.387281,2.410704,0.0,0.000000,0.000000,0.000000
75,7.679880,1.795303,5.313456,5.910707,5.403834,4.078740,2.384913,5.637014,0.933163,0.000000,...,1.028568,4.929844,5.521676,0.275970,0.138589,0.000000,0.0,4.296515,0.794905,0.000000


In [26]:
from sklearn.feature_selection import SelectFromModel
scaler = StandardScaler()
scaler.fit(X_train)

StandardScaler()

In [40]:

sel_ = SelectFromModel(
    LogisticRegression(C=0.9, penalty='l1', solver='liblinear', random_state=0))

sel_.fit((X_train), y_train)

SelectFromModel(estimator=LogisticRegression(C=0.9, penalty='l1',
                                             random_state=0,
                                             solver='liblinear'))

In [41]:
# the number of features which coefficient was shrank to zero:
np.sum(sel_.estimator_.coef_ == 0)

1114

In [42]:
# we can then remove the features from the training and testing set
# like this:

X_train_lasso = sel_.transform(X_train)
X_test_lasso = sel_.transform(X_test)

X_train_lasso.shape, X_test_lasso.shape

((63, 27), (27, 27))

In [43]:
feat_names_lasso = X_train.columns[sel_.get_support()]
feat_names_lasso

Index(['POU3F1', 'PPIAL4A', 'TEX50', 'OR2AK2', 'POU3F2', 'PAPOLB', 'TRGJ2',
       'TRBV7-6', 'NPM2', 'MSC', 'LYNX1', 'PKP3', 'OR5AK2', 'TRAV24',
       'TRAV38-1', 'TRAJ54', 'TRAJ52', 'TRAJ47', 'TRAJ44', 'TRAJ33', 'TRAJ30',
       'TRAJ28', 'KRT19', 'GAL3ST1', 'ADM2', 'NUDT11', 'DDX3Y'],
      dtype='object')

In [44]:
X_train_lasso = pd.DataFrame(X_train_lasso,index= X_train_uni.index, columns=feat_names_lasso)
X_train_lasso.head()

,POU3F1,PPIAL4A,TEX50,OR2AK2,POU3F2,PAPOLB,TRGJ2,TRBV7-6,NPM2,MSC,...,TRAJ47,TRAJ44,TRAJ33,TRAJ30,TRAJ28,KRT19,GAL3ST1,ADM2,NUDT11,DDX3Y
79,2.829794,2.953516,0.470608,0.823247,4.561627,0.136488,0.000000,0.000000,3.461918,6.200220,...,0.000000,6.890718,7.536333,0.000000,0.000000,3.013565,0.000000,0.000000,0.341195,8.110795
82,3.408138,2.570994,0.000000,0.498817,3.933324,2.645611,0.000000,0.000000,4.333843,5.326172,...,5.299356,7.729725,8.667147,1.314934,1.469976,0.795437,1.936706,0.000000,2.744351,8.199987
3,4.907083,0.000000,0.000000,4.945985,5.374499,4.089484,0.000000,0.000000,2.126141,6.039851,...,0.000000,7.125752,9.848754,7.271594,8.052657,7.093954,0.515203,1.462325,0.000000,1.632556
62,0.000000,0.000000,1.879338,4.913928,3.976795,0.000000,6.629023,5.979992,3.572991,6.061516,...,7.430117,9.277213,10.008092,8.425928,9.209154,2.612851,0.309973,0.985771,1.298078,0.000000
75,4.832066,0.000000,0.909888,3.737281,2.701823,2.663947,0.000000,0.000000,1.244932,5.110941,...,2.854111,2.804715,8.340896,7.331127,3.118381,1.468197,3.506076,1.436108,1.623877,4.296515


In [45]:
X_test_lasso = pd.DataFrame(X_test_lasso,index= X_test_uni.index, columns=feat_names_lasso)
X_test_lasso.head()

,POU3F1,PPIAL4A,TEX50,OR2AK2,POU3F2,PAPOLB,TRGJ2,TRBV7-6,NPM2,MSC,...,TRAJ47,TRAJ44,TRAJ33,TRAJ30,TRAJ28,KRT19,GAL3ST1,ADM2,NUDT11,DDX3Y
2,0.000000,4.424822,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.638431,5.413103,...,7.918793,9.353148,10.721029,7.918793,7.704738,0.000000,0.000000,0.000000,2.838338,8.503465
13,1.526903,0.000000,0.0,0.000000,5.772714,1.847559,6.843250,4.996727,0.000000,6.564119,...,8.227768,9.078777,9.961361,7.645210,8.013528,1.756107,1.079826,2.142982,1.824742,8.616417
53,1.289789,0.000000,0.0,5.510117,0.000000,1.997666,0.000000,4.073196,4.204518,6.736581,...,7.848204,8.436225,9.259323,7.848204,8.047418,0.000000,1.649923,2.643255,0.723144,8.314458
41,3.825500,0.000000,0.0,0.000000,6.339311,0.000000,0.000000,0.000000,1.013887,5.009920,...,0.000000,7.720997,0.000000,0.000000,0.000000,1.916603,0.000000,2.659276,2.309340,5.582638
66,5.245814,2.750574,0.0,0.000000,1.730919,1.464427,6.286523,6.723752,4.947670,5.979386,...,8.885685,9.515371,10.076614,8.663801,9.321952,0.000000,0.461562,2.279588,1.102683,0.000000


# lasso performance evaluation

In [49]:

clf1 = ExtraTreesClassifier()
clf2 = LogisticRegression(penalty ='l2', C=0.1, solver='liblinear', random_state=4,max_iter = 1000)
clf3 = SVC(gamma =1e-05, probability = True, decision_function_shape = 'ovo', kernel = 'linear')
clf4 = AdaBoostClassifier()
clf5= XGBClassifier()            
               
eclf1 = VotingClassifier(estimators=[  ('logistic', clf2),('SVM', clf3),
                                        ('XGB', clf5), ('extratree',clf1), 
                                        ('AdaBoost', clf4)
                                        ], voting='hard') 




for clf in (clf1,clf2, clf3, clf4, clf5, eclf1):
    clf.fit(X_train_lasso, y_train)
    y_pred = clf.predict(X_test_lasso)
    print(clf.__class__.__name__, round(accuracy_score(y_test, y_pred),3))



eclf1.fit(X_train_lasso, y_train)
predictions = eclf1.predict(X_test_lasso)
print(classification_report(y_test, predictions))

confusion_mat = confusion_matrix(y_test, predictions)
# Extract values from the confusion matrix
true_positive = confusion_mat[1, 1]
false_positive = confusion_mat[0, 1]
true_negative = confusion_mat[0, 0]
false_negative = confusion_mat[1, 0]

# Calculate Sensitivity (True Positive Rate)
sensitivity = true_positive / (true_positive + false_negative)

# Calculate Specificity
specificity = true_negative / (true_negative + false_positive)

# Print the results
print(f'Sensitivity (True Positive Rate): {sensitivity:.2f}')
print(f'Specificity: {specificity:.2f}')


ExtraTreesClassifier 0.778
LogisticRegression 0.852
SVC 0.852
AdaBoostClassifier 0.667
XGBClassifier 0.63
VotingClassifier 0.741
              precision    recall  f1-score   support

           0       0.79      0.79      0.79        14
           1       0.77      0.77      0.77        13

    accuracy                           0.78        27
   macro avg       0.78      0.78      0.78        27
weighted avg       0.78      0.78      0.78        27

Sensitivity (True Positive Rate): 0.77
Specificity: 0.79


# Select Features by Random Forests Importance

In [67]:
# use of the data selected by Univariate ROC-AUC Performance
X_train = X_train_uni
X_test = X_test_uni

In [68]:
# ensure the sizes of X_train and X_test.
X_train.shape, X_test.shape

((63, 1141), (27, 1141))

In [69]:
X_train_uni.head()

,MRPL20,PLCH2,CEP104,VAMP3,PIK3CD,PIK3CD-AS1,CTNNBIP1,DFFA,TMEM51-AS1,PADI3,...,PAK3,MCTS1,ZNF449,ADGRG4,FGF13,SPANXB1,GPR50,DDX3Y,LINC01238,PLA2G2C
79,7.570243,1.821964,5.856766,5.932988,5.742181,5.034410,1.973813,5.702246,0.973665,0.042752,...,0.151941,5.211416,5.415581,0.082789,0.137672,4.049688,0.0,8.110795,0.253986,0.274204
82,7.093483,1.173003,5.383863,5.930621,5.519404,5.176408,1.545476,5.867246,1.028031,1.085278,...,0.593874,4.935200,5.107300,0.152173,0.153646,2.556665,0.0,8.199987,0.171713,0.000000
3,7.192808,5.522151,5.474071,5.522044,4.459219,6.050044,2.908931,5.054147,1.955722,0.000000,...,0.979523,4.647634,5.297447,0.289800,0.390714,0.000000,0.0,1.632556,1.851140,0.000000
62,7.382451,2.226904,5.904765,5.805884,5.167068,7.019796,1.433969,5.277627,0.654643,0.000000,...,0.577084,4.519005,5.120259,0.000000,0.387281,2.410704,0.0,0.000000,0.000000,0.000000
75,7.679880,1.795303,5.313456,5.910707,5.403834,4.078740,2.384913,5.637014,0.933163,0.000000,...,1.028568,4.929844,5.521676,0.275970,0.138589,0.000000,0.0,4.296515,0.794905,0.000000


In [70]:
# select features using the impotance derived from
# random forests

from sklearn.feature_selection import SelectFromModel

sel_ = SelectFromModel(RandomForestClassifier(n_estimators=10, random_state=42))
sel_.fit(X_train, y_train)



SelectFromModel(estimator=RandomForestClassifier(n_estimators=10,
                                                 random_state=42))

In [71]:
# remove features with zero coefficient from dataset and parse again as dataframe (output of sklearn is numpy array)

X_train_rf = pd.DataFrame(sel_.transform(X_train),index= X_train.index, columns= X_train.columns[(sel_.get_support())])
X_test_rf = pd.DataFrame(sel_.transform(X_test),index= X_test.index, columns= X_train.columns[(sel_.get_support())])

In [72]:
X_train_rf.shape, X_test_rf.shape

((63, 50), (27, 50))

In [73]:
X_train_rf.columns

Index(['H3C14', 'OR2AK2', 'PTRHD1', 'TGFA', 'CTLA4', 'COL4A4', 'GBE1',
       'ATP8A1', 'FBXL7', 'GAPT', 'ERAP2', 'ADAM19', 'ZNF354C', 'PEX3',
       'HOXA4', 'TRGJ2', 'EGFR', 'HILPDA', 'TRBV7-6', 'ACTR3C', 'IDO2',
       'LYNX1', 'NLRP6', 'ESAM', 'NINJ2-AS1', 'IRAK4', 'LLPH', 'TRAV38-2DV8',
       'TRAJ52', 'TRAJ47', 'TRAJ33', 'TRAJ9', 'GCH1', 'ZNF280D', 'NPW',
       'VKORC1', 'MED1', 'TMEM106A', 'TBX21', 'CD7', 'NDUFV2-AS1', 'FSD1',
       'ZNF420', 'MYLK2', 'RBM12', 'RNF114', 'FNDC11', 'TMPRSS3', 'TEF',
       'EFNB1'],
      dtype='object')

In [74]:
X_train = X_train_rf
X_test= X_test_rf


# RF importance performance evaluation

In [75]:

clf1 = ExtraTreesClassifier()
clf2 = LogisticRegression(penalty ='l2', C=0.1, solver='liblinear', random_state=4,max_iter = 1000)
clf3 = SVC(gamma =1e-05, probability = True, decision_function_shape = 'ovo', kernel = 'linear')
clf4 = AdaBoostClassifier()
clf5= XGBClassifier()            
               
eclf1 = VotingClassifier(estimators=[  ('logistic', clf2),('SVM', clf3),
                                        ('XGB', clf5), ('extratree',clf1), 
                                        ('AdaBoost', clf4)
                                        ], voting='hard') 




for clf in (clf1,clf2, clf3, clf4, clf5, eclf1):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, round(accuracy_score(y_test, y_pred),3))



eclf1.fit(X_train, y_train)
predictions = eclf1.predict(X_test)
print(classification_report(y_test, predictions))

confusion_mat = confusion_matrix(y_test, predictions)
# Extract values from the confusion matrix
true_positive = confusion_mat[1, 1]
false_positive = confusion_mat[0, 1]
true_negative = confusion_mat[0, 0]
false_negative = confusion_mat[1, 0]

# Calculate Sensitivity (True Positive Rate)
sensitivity = true_positive / (true_positive + false_negative)

# Calculate Specificity
specificity = true_negative / (true_negative + false_positive)

# Print the results
print(f'Sensitivity (True Positive Rate): {sensitivity:.2f}')
print(f'Specificity: {specificity:.2f}')


ExtraTreesClassifier 0.852
LogisticRegression 0.852
SVC 0.815
AdaBoostClassifier 0.852
XGBClassifier 0.741
VotingClassifier 0.815
              precision    recall  f1-score   support

           0       0.80      0.86      0.83        14
           1       0.83      0.77      0.80        13

    accuracy                           0.81        27
   macro avg       0.82      0.81      0.81        27
weighted avg       0.82      0.81      0.81        27

Sensitivity (True Positive Rate): 0.77
Specificity: 0.86
